In [1]:
%pip install faiss-cpu sentence_transformers transformers datasets

Note: you may need to restart the kernel to use updated packages.


In [2]:
from datasets import load_dataset

agnews = load_dataset("ag_news")

agnews_train, agnews_valid = (
    agnews["train"].train_test_split(test_size=7600, seed=43).values()
)


Using custom data configuration default
Reusing dataset ag_news (/home/ufuk/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)


  0%|          | 0/2 [00:00<?, ?it/s]

Loading cached split indices for dataset at /home/ufuk/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548/cache-0fb8fdf64cc3fcc4.arrow and /home/ufuk/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548/cache-80423eae5320d3cb.arrow


In [6]:
import argilla as rg

In [7]:
agnews_data = rg.load("ag_news")

In [9]:
len(agnews_data)

9376

In [ ]:
# build our labelled records to evaluate our heuristic rules and optimize the thresholds
records = [
    rg.TextClassificationRecord(
        text=record["text"],
        metadata={"split": "labelled"},
        annotation=agnews_valid.features["label"].int2str(record["label"]),
        id=f"valid_{idx}",
    )
    for idx, record in enumerate(agnews_valid)
]

# build our unlabelled records
records += [
    rg.TextClassificationRecord(
        text=record["text"],
        metadata={"split": "unlabelled"},
        id=f"train_{idx}",
    )
    for idx, record in enumerate(agnews_train)
]

# log the records to Argilla
rg.log(records, name="news2")


In [ ]:
def record_generator(hf_dataset, model, split, vector_name, batch_size=32):
    record_batch = []
    for idx, record in enumerate(hf_dataset):
        vector = model.encode(record["text"]).tolist()
        record = rg.TextClassificationRecord(
            text=record["text"],
            embeddings = {
                vector_name: {
                    "vector": vector
                }
            },
            metadata={"split": split},
            id=f"train_{idx}",
        )
        record_batch.append(record)
        if len(record_batch) == batch_size:
            yield record_batch
            record_batch = []
    if record_batch:
        yield record_batch